# Data Scraping
### Since for this task we need good amout of data. I had tried to scrap it using Beautiful soup but have some problem as we are geting less instances and some of the information was unretrievable. So I used Reddit API for this, which gives a flexible api to retrive data based on limits.
I do used Beautiful soup for data extraion in Web app. (For demo purpose)
***
## Reference

- [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Reddit API](https://praw.readthedocs.io/en/latest/getting_started/configuration/prawini.html)

### We have used these major Flairs for classification.

In [47]:
flairs_list = ["AskIndia", 
          "Non-Political", 
          "Policy/Economy",
          "Politics", 
          "Photography",
          "Business/Finance",
          "Science/Technology",
          "Sports", 
          "Scheduled",
          "Food", 
          "Coronavirus",
          "AMA"]

### I have created this dictionary so that it will be easily converted in Dataframe

In [55]:
# Flair - List of all Post Flair
# Title - List of all Post Title
# score - List of all scores of respective post
# full_link - List of all Post Url addresss
# selftext - List of all Post Body
# Author - List of all Post Author
# combined_comments - List of top 5 cascaded comments

flair_dictionary = { "Flair":[],
                     "Title":[],
                     "Created_at":[],
                     "Score":[],
                     "Url_address":[],
                     "Body":[],
                     "Author":[],
                     "num_comments":[],
                     "combined_comments":[]}

#### Initializing `Reddit API`

In [ ]:
# Importing Library Reddit Library
import praw
# Initializing Reddit API
reddit = praw.Reddit(client_id='KhELuATyIFR1aQ',
                     client_secret='gHm5vxbsctlwSE8yqtpLKAdFR7E',
                     user_agent='ahmadkhan',
                        username='ahmadkhan242', 
                     password='9990800652')
print(reddit.user.me())

# Extracting submission respective above chosen flairs.
<h4>Here we are extracting same no of post for each flair we defined in the above cell. Here we use search option given by reddit API to search most relevent data for give flair.<h4/>

In [57]:
from tqdm import tqdm

# Choosing India as subrediit
subreddit = reddit.subreddit('india')
# Iterating over flairs_list

for flair in flairs_list:
    collect_subreddit = subreddit.search (flair, limit=None)
    with tqdm(total=300, desc=flair) as pbar:
        for i in collect_subreddit:
            flair_dictionary["Flair"].append (flair)
            flair_dictionary["Title"].append (i.title)
            flair_dictionary["Created_at"].append (i.created)
            flair_dictionary["Score"].append (i.score)
            flair_dictionary["Url_address"].append(i.url)
            flair_dictionary["Body"].append(i.selftext)
            flair_dictionary["Author"].append (i.author)
            flair_dictionary["num_comments"].append (i.num_comments)
            i.comments.replace_more(limit=None)
            combined_comments = " "
            c = 0
            try:
                for comment in i.comments:
                    combined_comments += " " + comment.body
                    c+=1
                    if c>11 :
                        break
            except:
                combined_comments = combined_comments
            flair_dictionary["combined_comments"].append(combined_comments)
            pbar.update(1)
        if(flair_dictionary['Flair'].count(flair) >= 300):
            break
            


AMA:  71%|███████   | 212/300 [34:38<14:22,  9.80s/it]   


### Now using Pandas we convert `flair_dictionary` in dataframe and save the DataFrame as CSV file.

In [58]:
# Importing Pandas
import pandas as pd

# Creating DataFrame from flairs_dictionary 
my_data = pd.DataFrame(flair_dictionary)

# Saveing above DataFrame as .csv file on Local
my_data.to_csv('flair_dataset.csv', index = False)

# Why to choose similar number of post for each Flair?
Post on Reddit are not evenly distribueted based on flairs. Post appears in random basis sometime they change with time like if we take flairs for this year till now, Coronavirus flair appears more than photograpy. So if we take data randomly it is possible that we get more data for one flair than another, this will lead the model to overfit on one flair than all of them. So for unbaise classification we take same number of post data for each Flair.  
```Analysis of data based on date of Submission is in EDA section to validate this claim.```  
[Refrence 1](https://www.analyticsvidhya.com/blog/2017/03/imbalanced-data-classification/)  
[Refrence 2](https://elitedatascience.com/imbalanced-classes)

## Small Data extraction using BeautifulSoup.

In [48]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.reddit.com/r/india/comments/fxo4xy/surveillance_drone_in_my_redzone/'
headers ={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36"}
with requests.Session() as s:
    r = s.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "lxml")
    Titles = soup.find_all('h1', class_="_eYtD2XCVieq6emjKBH3m")
    Flair_div = soup.find_all('div', class_="_2X6EB3ZhEeXCh1eIVA64XM")
    Flair = Flair_div[0].select_one("span")
    print("Title = ", Titles[0].text)
    print("Flair = ", Flair.text)

Title =  Surveillance drone in my red-zone
Flair =  Coronavirus
